In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [4]:
TEST_CASE_GENERATOR_PROMPT="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [5]:
TEST_CASE_VALIDATOR="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [6]:
scenario =  """7. Attempt to add a product with no stock"""

In [7]:
usecase_content = """
Add item to cart
Description
This feature allows users to add items to their shopping cart for future purchase.
Actor: User
Preconditions: User is logged into their Shopee account.
Postconditions: 
The item is successfully added to the user's shopping cart.
The user can view and manage items in their shopping cart.
Basic Flow:
Step 1: User perform feature 'View a product detail'.
Step 2: User clicks the "Add to Cart" button.
Step 3: System confirms that the item has been added to the cart.
Step 5: System adds item to the cart and updates the cart icon to reflect the number of items in the cart.
Alterative flow: 
If product has many options, user choose one available option before adding product to cart.
User can adjust quantity of product before adding product into cart by clicking on minus or plus button next to quantity of the product. 

Exception flow: 
user cannot add product has many options without selecting one available option.
user cannot add product with quantity that is bigger than the current stock. If user tries to adjust the quantity over the maximum stock, the plus button is disabled and a message is shown bellow the quantity user tries to add "You has reach maximum of the stock."
If the quantity is at one, the minus button is disabled to prevent user adjust quantity below one. 
User cannot add to cart product with no stock, or out of stock option of one product with multiple options.
"""

In [8]:

promptTestCaseGenerator = [
{ "role": "system", "content": TEST_CASE_GENERATOR_PROMPT},
{ "role": "user", "content": scenario + "\n" + usecase_content}
]
gpt_response = ask(promptTestCaseGenerator, client, model)
json_data = json.loads(gpt_response)
print(gpt_response)
promptTestCaseValidate = [
    { "role": "system", "content": TEST_CASE_VALIDATOR},
    { "role": "user", "content": scenario + "\n" + gpt_response}
]
final_response = ask(promptTestCaseValidate, client, model)
json_fin = json.loads(final_response)
print(final_response)

{
  "Test Case 1": {
    "testCaseName": "Attempt to add a product with no stock to the shopping cart",
    "objective": "Verify that a user cannot add a product with no stock to their shopping cart and receives an appropriate message indicating the product is out of stock.",
    "testSteps": [
      "Step 1: Log into the Shopee account.",
      "Step 2: Navigate to a product detail page for a product with no stock.",
      "Step 3: Click the 'Add to Cart' button."
    ],
    "expectedResult": "The system should display a message indicating the product is out of stock and prevent the item from being added to the shopping cart.",
    "explanation": "This test case ensures that users are informed when they attempt to add an out-of-stock product to their cart, preventing confusion and ensuring a good user experience by adhering to the exception flow where users cannot add to cart product with no stock."
  },
  "Test Case 2": {
    "testCaseName": "Attempt to add an out-of-stock option of 